## Tutoriel : interagir avec le système de stockage S3 du SSP Cloud (MinIO)

In [2]:
import os

import pandas as pd
import s3fs
import zipfile

### Récupérer les données d'un challenge

In [3]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [4]:
# Lister les challenges
fs.ls("gvimont/diffusion/hackathon-minarm-2024")

['gvimont/diffusion/hackathon-minarm-2024/AIVSAI',
 'gvimont/diffusion/hackathon-minarm-2024/Acoustique',
 'gvimont/diffusion/hackathon-minarm-2024/Similarité']

In [5]:
# Lister les fichiers d'un challenge
fs.ls("gvimont/diffusion/hackathon-minarm-2024/Acoustique")

['gvimont/diffusion/hackathon-minarm-2024/Acoustique/LivingRoom_preprocessed_hack.zip']

In [7]:
# Télécharger les données dans le service
PATH_IN = 'gvimont/diffusion/hackathon-minarm-2024/Acoustique/LivingRoom_preprocessed_hack.zip'
fs.download(PATH_IN, 'data/LivingRoom_preprocessed_hack.zip')

[None]

In [8]:
# Décompresser les données
with zipfile.ZipFile("data/LivingRoom_preprocessed_hack.zip","r") as zip_file:
    zip_file.extractall("data/")

In [10]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, InputLayer

# Load the RIRs data
data_rirs = np.load('data/LivingRoom_preprocessed_hack/Human1/deconvoled_trim.npy')

# Load the centroid data (positions of humans)
data_centroid = np.load('data/LivingRoom_preprocessed_hack/Human1/centroid.npy')

# Reshape the RIRs data to fit the model's input requirements
data_rirs = data_rirs.reshape(-1, 4, data_rirs.shape[2], 1)  # Adding the channel dimension

# Define the VGG-like model for multi-channel RIR data
model = Sequential([
    InputLayer(input_shape=(4, data_rirs.shape[2], 1)),  # Adjusted based on data shape
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), strides=(1, 2)),  # Reduced stride on the frequency axis
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), strides=(1, 2)),  # Reduced stride on the frequency axis
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2), strides=(1, 2)),  # Reduced stride on the frequency axis
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(2)  # Output layer with 2 neurons for x, y coordinates
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


model.fit(data_rirs, data_centroid, epochs=10, batch_size=32, validation_split=0.2)

test_rirs = np.load('path/to/test_rirs.npy').reshape(-1, 4, test_rirs.shape[2], 1)
test_centroid = np.load('path/to/test_centroid.npy')

loss = model.evaluate(test_rirs, test_centroid)
print("Test Loss:", loss)

predictions = model.predict(test_rirs)
print("Predictions:", predictions)

/opt/mamba/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
2024-04-23 16:35:26.129989: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13290 MB memory:  -> device: 0, name: NVIDIA A2, pci bus id: 0000:ca:00.0, compute capability: 8.6
2024-04-23 16:35:36.815989: W external/local_tsl/tsl/framework/bfc_allocator.cc:487] Allocator (GPU_0_bfc) ran out of memory trying to allocate 81.45GiB (rounded to 87451238400)requested by op StatelessRandomUniformV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-04-23 16:35:36.816072: I external/local_tsl/tsl/framework/bfc_allocator.cc:1044] BFCAllocator dump for GPU_0_bfc
2024-04-23 16:35:36.816104: I external/

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[21350400,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2] name: 